In [6]:
import geopandas as gpd
import numpy as np
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from ipyleaflet import Map, GeoJSON, LayersControl, basemaps
from ipywidgets import Layout, VBox, HBox
import ipywidgets as widgets

def create_gradient_legend(cmap, min_val, max_val):
    """Generates an HTML legend for the colormap."""
    rgb_values = [mpl.colors.rgb2hex(cmap(i)) for i in np.linspace(0, 1, 256)]
    gradient_style = ", ".join(rgb_values)
    legend_html = f"""
<div style="display: flex; flex-direction: column; align-items: center;">
    <div style="
        background: linear-gradient(to right, {gradient_style});
        height: 30px;
        width: 200px;
        border: 1px solid black;
    "></div>
    <div style="text-align: center; width: 100%;">
        <span>Low: {min_val:.1f}</span> | <span>High: {max_val:.1f}</span>
    </div>
</div>
"""
    return widgets.HTML(legend_html)

# Read GeoJSON files
gdf_line = gpd.read_file("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/station2grid_line.geojson")
gdf_station = gpd.read_file("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/station.geojson")
gdf_grid = gpd.read_file("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/grid_point.geojson")

# Define a colormap (using Matplotlib)
cmap = plt.cm.get_cmap('PRGn')

# Normalize the "area_difference" column for the color mapping
min_val = - 1000
max_val = 1000
# min_val = gdf_line['area_difference'].min()
# max_val = gdf_line['area_difference'].max()
norm = plt.Normalize(vmin=min_val, vmax=max_val)
legend_widget = create_gradient_legend(cmap, min_val, max_val)

def style_callback(feature):
    """Style function for the GeoJSON layer based on 'area_difference'."""
    area_diff = feature['properties']['area_difference']
    color = mcolors.to_hex(cmap(norm(area_diff))) if area_diff is not None else "#ffffff"
    return {'color': color, 'weight': 2}

# Create a Map instance
m = Map(basemap=basemaps.CartoDB.Positron,center=(0, 0), zoom=2, layout=Layout(height='600px'))

# Add styled GeoJSON layer for lines
geojson_line = GeoJSON(
    data=gdf_line.__geo_interface__, 
    style_callback=style_callback, 
    name="Area Difference"
)
m.add_layer(geojson_line)

# Style for station points
station_style = {
    'radius': 5,
    'color': 'red',
    'fillColor': 'red',
    'fillOpacity': 1
}

# Add GeoJSON layer for stations
geojson_station = GeoJSON(
    data=gdf_station.__geo_interface__, 
    point_style=station_style, 
    name="Stations"
)
m.add_layer(geojson_station)

# Style for station points
grid_style = {
    'radius': 5,
    'color': 'blue',
    'fillColor': 'blue',
    'fillOpacity': 0
}

# Add GeoJSON layer for stations
geojson_grid = GeoJSON(
    data=gdf_grid.__geo_interface__, 
    point_style=grid_style, 
    name="Grid Points"
)
m.add_layer(geojson_grid)


# Add layers control
m.add_control(LayersControl())

# Display the map and legend in a vertical layout
display(widgets.VBox([m, legend_widget]))


/tmp/ipykernel_30664/3893591872.py:35: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('PRGn')


In [ ]:
gdf_line